# Day 16

In [1]:
import pandas as pd
import numpy as np
import sys
#np.set_printoptions(threshold=sys.maxsize)

In [2]:
with open('input_files/day16_input.txt') as f:
    lines = f.read()

In [3]:
lines

'E20D79005573F71DA0054E48527EF97D3004653BB1FC006867A8B1371AC49C801039171941340066E6B99A6A58B8110088BA008CE6F7893D4E6F7893DCDCFDB9D6CBC4026FE8026200DC7D84B1C00010A89507E3CCEE37B592014D3C01491B6697A83CB4F59E5E7FFA5CC66D4BC6F05D3004E6BB742B004E7E6B3375A46CF91D8C027911797589E17920F4009BE72DA8D2E4523DCEE86A8018C4AD3C7F2D2D02C5B9FF53366E3004658DB0012A963891D168801D08480485B005C0010A883116308002171AA24C679E0394EB898023331E60AB401294D98CA6CD8C01D9B349E0A99363003E655D40289CBDBB2F55D25E53ECAF14D9ABBB4CC726F038C011B0044401987D0BE0C00021B04E2546499DE824C015B004A7755B570013F2DD8627C65C02186F2996E9CCD04E5718C5CBCC016B004A4F61B27B0D9B8633F9344D57B0C1D3805537ADFA21F231C6EC9F3D3089FF7CD25E5941200C96801F191C77091238EE13A704A7CCC802B3B00567F192296259ABD9C400282915B9F6E98879823046C0010C626C966A19351EE27DE86C8E6968F2BE3D2008EE540FC01196989CD9410055725480D60025737BA1547D700727B9A89B444971830070401F8D70BA3B8803F16A3FC2D00043621C3B8A733C8BD880212BCDEE9D34929164D5CB08032594E5E1D25C0055E5B771E966783240220CD19E8

In [4]:
import unittest, functools

# Read puzzle input and return as usable data structure
def parse_input(file):
    hex_vals = []
    with open(file, 'r') as input:
        for line in input:
            hex_vals.extend([char for char in line.rstrip()])
    return hex_vals

# Find value of a literal value subpacket
def get_lit_val(packet):
    lit_val = ""
    while packet[0] == "1":
        lit_val += "".join(packet[1:5])
        packet = packet[5:]
    lit_val += "".join(packet[1:5])
    packet = packet[5:]
    return int(lit_val, 2), packet

# Recursive function to decode packets and subpackets
def pack_decode(packet):
    version, typeID, packet = int(packet[:3], 2), int(packet[3:6], 2), packet[6:]
    value = 0

    if typeID == 4:
        value, packet = get_lit_val(packet)
    else:
        lengthID, packet = int(packet[0], 2), packet[1:]
        literals = []
        if lengthID == 0:
            # If length type is 0, keep converting subpackets for length given
            length, packet = int(packet[:15], 2), packet[15:]
            sub_packet, packet = packet[:length], packet[length:]
            while sub_packet:
                sub_version, sub_value, sub_packet = pack_decode(sub_packet)
                version += sub_version
                literals.append(sub_value)
        else:
            # If length type is 1, convert x number of subpackets
            pack_num, packet = int(packet[:11], 2), packet[11:]
            for _ in range(pack_num):
                sub_version, sub_value, packet = pack_decode(packet)
                version += sub_version
                literals.append(sub_value)
        if typeID == 0:
            value = sum(literals)
        elif typeID == 1:
            value = functools.reduce(lambda x, y: x * y, literals)
        elif typeID == 2:
            value = min(literals)
        elif typeID == 3:
            value = max(literals)
        elif typeID == 5:
            value = 1 if literals[0] > literals[1] else 0
        elif typeID == 6:
            value = 1 if literals[0] < literals[1] else 0
        elif typeID == 7:
            value = 1 if literals[0] == literals[1] else 0

    return version, value, packet

def packet_decoder(file, result):
    hex_vals = parse_input(file)

    # Convert hex value to decimal, then to binary
    packet = f'{int("".join(hex_vals), 16):0>{4 * len(hex_vals)}b}'
    total_ver, total_val, packet = pack_decode(packet)

    # result = 0 for part 1
    # result = 1 for part 2
    if result == 0:
        return total_ver
    elif result == 1:
        return total_val

if __name__ == "__main__":
    #unittest.main(verbosity=2)
    
    # Part 1 solution
    #print(packet_decoder('Advent of Code/test.txt', 0))
    print(packet_decoder('input_files/day16_input.txt', 0))
    
    # Part 2 solution
    #print(packet_decoder('Advent of Code/test.txt', 1))
    print(packet_decoder('input_files/day16_input.txt', 1))

951
902198718880


In [5]:
### Older versions / draft
def convert_hexa_to_bin(hexa_number):
    map_hexa = {"0":"0000","1":"0001","2":"0010","3":"0011","4":"0100","5":"0101","6":"0110",
               "7":"0111","8":"1000","9":"1001","A":"1010","B":"1011","C":"1100","D":"1101","E":"1110","F":"1111"}
    bin_number=""
    for char in hexa_number:
        bin_number+=map_hexa[char]
    return bin_number

def get_packet_version_and_id(bin_number):
    packet_version = int(bin_number[0:3],2)
    packet_id = int(bin_number[3:6],2)
    return packet_version, packet_id

def packet_is_literal(bin_number):
    packet_version, packet_id = get_packet_version_and_id(bin_number)
    if packet_id == 4:
        return True
    else:
        return False

def unpack_literal_value(bin_number):
    packet_version, packet_id = get_packet_version_and_id(bin_number)
    five_bits_groups = bin_number[6:]
    first_number = five_bits_groups[0]
    final_number = ""
    is_end = False
    while is_end == False:
        first_number = five_bits_groups[0]
        final_number+=five_bits_groups[1:5]
        five_bits_groups = five_bits_groups[5:]
        is_end = True if first_number=="0" else False
    remaining_string = bin_number.replace(final_number,'')
    return packet_version, int(final_number,2),final_number, remaining_string

def unpack_operator_packet(bin_number):
    packet_version, packet_id = get_packet_version_and_id(bin_number)
    length_type_id = int(bin_number[6], 2)
    bit_length = 15 if length_type_id == 0 else 11
    print("operator version:", packet_version, ", bit lenght", bit_length, ", bin_number", bin_number)
    #Mode 1
    
    if bit_length == 15:
        subpackets_length = int(bin_number[7:7+bit_length], 2)
        scanned_nrs = int(0)
        start_subpackets = 7+bit_length
        remaining_subpackets = bin_number[start_subpackets:start_subpackets+subpackets_length]
        print("subpacket length", subpackets_length)
        #print("start loop scanned nrs", scanned_nrs)

        if packet_is_literal(remaining_subpackets):
            while scanned_nrs < subpackets_length:
                print("remaining literal", remaining_subpackets)
                packet_version, packet_id = get_packet_version_and_id(remaining_subpackets) #get first packet version and id

                print("literal found inside 15 bits operator, version:", unpack_literal_value(remaining_subpackets)[0], "number", unpack_literal_value(remaining_subpackets)[1])
                len_full_literal = int(len(unpack_literal_value(remaining_subpackets)[2])+6+len(unpack_literal_value(remaining_subpackets)[2])/4)
                scanned_nrs +=  len_full_literal
                print("scanned nrs", scanned_nrs)
                remaining_subpackets = bin_number[start_subpackets+scanned_nrs:start_subpackets+subpackets_length]
                    
            #After scanning all the literals returns rest of string
            remaining_subpackets = bin_number[start_subpackets+subpackets_length:]
            #return remaining_subpackets
        else:
            print("unpack_operator_packet inside 15 bits, version",get_packet_version_and_id(remaining_subpackets)[0])
            unpack_operator_packet(remaining_subpackets)
            scanned_nrs +=6+1+15
            print("unpack", scanned_nrs)
            #break
        #return remaining_subpackets = bin_number[start_subpackets+subpackets_length:]
    
    #Mode 2
    elif bit_length == 11:
        subpackets_number = int(bin_number[7:7+bit_length], 2)
        subpackets_found = 0
        start_subpackets = 7+bit_length
        remaining_subpackets = bin_number[start_subpackets:]
        print("number of subpackets",subpackets_number)
        while subpackets_found < subpackets_number:
            print("remaining", remaining_subpackets)
            packet_version, packet_id = get_packet_version_and_id(remaining_subpackets) #get first packet version and id
            
            if packet_is_literal(remaining_subpackets):
                print("literal found inside 11 bits operator, version:", unpack_literal_value(remaining_subpackets)[0], "number", unpack_literal_value(remaining_subpackets)[1])
                len_full_literal = int(len(unpack_literal_value(remaining_subpackets)[2])+6+len(unpack_literal_value(remaining_subpackets)[2])/4)
                subpackets_found +=  1
                print("subpackets_found", subpackets_found)
                remaining_subpackets = bin_number[start_subpackets+len_full_literal:]
            else:
                print("unpacking new operator inside packet 11 bits, version",get_packet_version_and_id(remaining_subpackets)[0])
                subpackets_found +=  1
                unpack_operator_packet(remaining_subpackets)
                remaining_subpackets = bin_number[start_subpackets+len_full_literal:]

                #break

        
#return length_type_id, bit_length,subpackets_length